In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import json
from tqdm.auto import tqdm
import numpy as np
import os

c:\Users\shahr\anaconda3\envs\lm-prompt-turbo2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Constants
# models = [
#     'microsoft/deberta-v3-xsmall',
#     'microsoft/deberta-v3-small',
#     'microsoft/deberta-v3-large',
#     'microsoft/deberta-v3-base',
# ]

MODEL_NAME = 'cross-encoder/nli-deberta-v3-base'
# MODEL_NAME = 'cross-encoder/nli-deberta-v3-small'
# MODEL_NAME = 'microsoft/deberta-v3-base'
# MODEL_NAME = "sileod/deberta-v3-base-tasksource-nli"
# MODEL_NAME = "clagator/biobert_v1.1_pubmed_nli_sts"
# MODEL_NAME = "gsarti/scibert-nli"
# MODEL_NAME = "gsarti/biobert-nli"
# MODEL_NAME = "dmis-lab/biobert-base-cased-v1.2" BAD
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [3]:
def debug_tokenization(model_name, primary_premise, secondary_premise, statement, premise_to_use, max_premise_length, max_input_length):
    # Initialize the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # if premise_to_use == "Original_truncated":
    #     # Tokenize the primary premise
    #     primary_tokens = tokenizer.encode(primary_premise, add_special_tokens=False)
    #     individual_premise_length = max_premise_length if not secondary_premise else int(max_premise_length / 2)
    #     combined_tokens = primary_tokens[:individual_premise_length]

    #     if secondary_premise:
    #         # Tokenize the secondary premise
    #         secondary_tokens = tokenizer.encode(secondary_premise, add_special_tokens=False)
    #         # combined_tokens += [tokenizer.sep_token_id] + secondary_tokens[:individual_premise_length]
    #         combined_tokens += secondary_tokens[:individual_premise_length]
    # else:  # summarized_premise
    #     # Tokenize the summarized premise
    #     combined_tokens = tokenizer.encode(primary_premise, add_special_tokens=False)

    if premise_to_use == "Combined":
        # Tokenize the summarized premise
        combined_tokens = tokenizer.encode(primary_premise, add_special_tokens=False)
    else:
        # Tokenize the primary premise
        primary_tokens = tokenizer.encode(primary_premise, add_special_tokens=False)
        individual_premise_length = max_premise_length if not secondary_premise else int(max_premise_length / 2)
        combined_tokens = primary_tokens[:individual_premise_length]

        if secondary_premise:
            # Tokenize the secondary premise
            secondary_tokens = tokenizer.encode(secondary_premise, add_special_tokens=False)
            # combined_tokens += [tokenizer.sep_token_id] + secondary_tokens[:individual_premise_length]
            combined_tokens += secondary_tokens[:individual_premise_length]

    # Tokenize the statement
    statement_tokens = tokenizer.encode(statement, add_special_tokens=False)

    # Combine tokens for final input
    input_tokens = [tokenizer.cls_token_id] + combined_tokens + [tokenizer.sep_token_id] + statement_tokens

    # Truncate to max_input_length if necessary
    input_tokens = input_tokens[:max_input_length - 1] + [tokenizer.sep_token_id]

    # Convert token IDs back to text
    tokenized_text = tokenizer.decode(input_tokens)

    # Print tokenized output as text
    print("Tokenized Text:", tokenized_text)

# Example usage
debug_tokenization(
    model_name="microsoft/deberta-v3-xsmall",
    primary_premise="Primary premise text here."*int(15),
    secondary_premise=None,  # Use None if not applicable
    statement="Statement text here.",
    premise_to_use="Original_truncated",  # Or "summarized_premise"
    max_premise_length=256,
    max_input_length=512
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
c:\Users\shahr\anaconda3\envs\lm-prompt-turbo2\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Tokenized Text: [CLS] Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.Primary premise text here.[SEP] Statement text here.[SEP]


In [4]:
# 1. Data Preprocessing
class NliDataset(Dataset):
    def __init__(self, data, tokenizer, premise_combined, primary_premise_to_use, secondary_premise_to_use, max_premise_length=256, max_input_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.premise_combined = premise_combined
        self.primary_premise_to_use = primary_premise_to_use
        self.secondary_premise_to_use = secondary_premise_to_use
        self.max_premise_length = max_premise_length
        self.max_input_length = max_input_length
        self.keys = list(data.keys())  # Store the keys of the dictionary

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]  # Use the index to get the corresponding key
        item = self.data[key]  # Retrieve the item using the key

        # Tokenize the statement and add [SEP] token at the beginning
        statement_tokens = self.tokenizer.encode(item["Statement"], add_special_tokens=False)
        len_statement_tokens = len(statement_tokens)

        remaining_for_premise_length = max(self.max_input_length - len_statement_tokens - 10, self.max_premise_length)

        if self.premise_combined:
            # Tokenize and use the summarized premise
            combined_tokens = self.tokenizer.encode(item[self.primary_premise_to_use], add_special_tokens=False)
        else:
            # Tokenize the primary premise
            primary_tokens = self.tokenizer.encode(item[self.primary_premise_to_use], add_special_tokens=False)
            
            individual_premise_length = int(remaining_for_premise_length) if item["Type"] != "Comparison" else int(remaining_for_premise_length / 2)
            combined_tokens = primary_tokens[:individual_premise_length]

            if item["Type"] == "Comparison":
                # Tokenize the secondary premise
                secondary_tokens = self.tokenizer.encode(item[self.secondary_premise_to_use], add_special_tokens=False)
                combined_tokens += secondary_tokens[:individual_premise_length]



        

        # Combine tokens for final input
        input_tokens = [self.tokenizer.cls_token_id] + combined_tokens + [self.tokenizer.sep_token_id] + statement_tokens

        # Truncate to max_input_length if necessary
        input_tokens = input_tokens[:self.max_input_length - 1] + [self.tokenizer.sep_token_id]

        # Convert to PyTorch tensors
        input_ids = torch.tensor(input_tokens).unsqueeze(0)
        attention_mask = torch.tensor([1] * len(input_tokens)).unsqueeze(0)

        return {
            "input_ids": input_ids.flatten(),
            "attention_mask": attention_mask.flatten(),
            "labels": torch.tensor(item["Label"] == "Entailment", dtype=torch.long)
        }

def preprocess_data(tokenizer, data, premise_combined, primary_premise_to_use, secondary_premise_to_use):
    return NliDataset(data, tokenizer, premise_combined, primary_premise_to_use, secondary_premise_to_use)

# 2. Model Setup
def get_model(model_name, device='cpu'):
    # model = AutoModel.from_pretrained(model_name, num_labels=2)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
    model.to(device)
    return model

# 3. Training and Evaluation
def train(model, train_loader, optimizer, lr_scheduler, device):
    model.train()
    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

def evaluate(model, data_loader, set_name, device):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(data_loader, desc=f"Evaluating on {set_name}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            preds = torch.argmax(F.log_softmax(logits, dim=1), dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(batch["labels"].cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions, average="macro", zero_division=0)
    return accuracy, f1, predictions, true_labels

def save_predictions(data, predictions, file_path):
    # Assume predictions is a list of integers (0 or 1)
    # Convert predictions to the corresponding label strings
    label_predictions = ["Entailment" if pred == 1 else "Contradiction" for pred in predictions]

    # Make a copy of the data to avoid modifying the original data
    updated_data = {key: dict(value, prediction=label_predictions[i]) for i, (key, value) in enumerate(data.items())}

    # Save the updated data with predictions to a file
    with open(file_path, 'w') as file:
        json.dump(updated_data, file, indent=4)

def save_model_and_optimizer(model, optimizer, file_path):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, file_path)

### Load Data

In [5]:
def get_premises_text(gold, raw_data_location):
  for data in gold:
        section = gold[data]["Section_id"]
        primary_id = gold[data]["Primary_id"]
        with open(f'{raw_data_location}/{primary_id}.json') as f:
            primary = json.load(f)

        primary_premise = ' '.join(primary[section])
        gold[data]["Primary_premise"] = primary_premise

        if gold[data]["Type"] == 'Comparison':
            secondary_id = gold[data]["Secondary_id"]
            with open(f'{raw_data_location}/{secondary_id}.json') as f:
                secondary = json.load(f)
            secondary_premise = ' '.join(secondary[section])
            gold[data]["Secondary_premise"] = secondary_premise

with open('data\\raw\\train.json') as f:
    train_data = json.load(f)
with open('data\\raw\\dev.json') as f:
    dev_data = json.load(f)
with open('data\\raw\\test.json') as f:
    test_data = json.load(f)

get_premises_text(train_data, 'data\\raw\\CT')
get_premises_text(dev_data, 'data\\raw\\CT')
get_premises_text(test_data, 'data\\raw\\CT')

#### Add Zero-Shot Summarizations and Fine-Tuned Summarizations

In [6]:
def update_data_dict(new_data_dict, old_data_dict, new_key, key_to_use):
    for key in new_data_dict:
        if key_to_use in new_data_dict[key]:
            old_data_dict[key][new_key] = new_data_dict[key][key_to_use]

In [7]:
for fine_tuning_steps in [0, 2, 5, 7, 10]:
    with open(f'data\\raw\\summary_train_{fine_tuning_steps}.json') as f:
        train_data_summarized = json.load(f)

    with open(f'data\\raw\\summary_dev_{fine_tuning_steps}.json') as f:
        dev_data_summarized = json.load(f)

    with open(f'data\\raw\\summary_test_{fine_tuning_steps}.json') as f:
        test_data_summarized = json.load(f)

    update_data_dict(train_data_summarized, train_data, f"Summarized_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(dev_data_summarized, dev_data, f"Summarized_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(test_data_summarized, test_data, f"Summarized_Primary_premise_{fine_tuning_steps}", "Primary_Premise")

    update_data_dict(train_data_summarized, train_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")
    update_data_dict(dev_data_summarized, dev_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")
    update_data_dict(test_data_summarized, test_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")

In [8]:
for fine_tuning_steps in [0, 2, 5, 7]:
    with open(f'data\\raw\\scifive_train_{fine_tuning_steps}.json') as f:
        train_data_summarized = json.load(f)

    with open(f'data\\raw\\scifive_dev_{fine_tuning_steps}.json') as f:
        dev_data_summarized = json.load(f)

    with open(f'data\\raw\\scifive_test_{fine_tuning_steps}.json') as f:
        test_data_summarized = json.load(f)

    update_data_dict(train_data_summarized, train_data, f"Scifive_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(dev_data_summarized, dev_data, f"Scifive_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(test_data_summarized, test_data, f"Scifive_Primary_premise_{fine_tuning_steps}", "Primary_Premise")

    update_data_dict(train_data_summarized, train_data, f"Scifive_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")
    update_data_dict(dev_data_summarized, dev_data, f"Scifive_Seconday_premise_{fine_tuning_steps}", "Secondary_Premise")
    update_data_dict(test_data_summarized, test_data, f"Scifive_Seconday_premise_{fine_tuning_steps}", "Secondary_Premise")

In [9]:
for fine_tuning_steps in [0]:
    with open(f'data\\raw\\combined_train_{fine_tuning_steps}.json') as f:
        train_data_summarized = json.load(f)

    with open(f'data\\raw\\combined_dev_{fine_tuning_steps}.json') as f:
        dev_data_summarized = json.load(f)

    with open(f'data\\raw\\combined_test_{fine_tuning_steps}.json') as f:
        test_data_summarized = json.load(f)

    update_data_dict(train_data_summarized, train_data, f"Combined_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(dev_data_summarized, dev_data, f"Combined_Primary_premise_{fine_tuning_steps}", "Primary_Premise")
    update_data_dict(test_data_summarized, test_data, f"Combined_Primary_premise_{fine_tuning_steps}", "Primary_Premise")

    # update_data_dict(train_data_summarized, train_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")
    # update_data_dict(dev_data_summarized, dev_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")
    # update_data_dict(test_data_summarized, test_data, f"Summarized_Secondary_premise_{fine_tuning_steps}", "Secondary_Premise")

In [10]:
train_data[list(train_data.keys())[0]]

{'Type': 'Comparison',
 'Section_id': 'Intervention',
 'Primary_id': 'NCT01928186',
 'Secondary_id': 'NCT00684983',
 'Statement': 'All the primary trial participants do not receive any oral capecitabine, oral lapatinib ditosylate or cixutumumab IV, in conrast all the secondary trial subjects receive these.',
 'Label': 'Contradiction',
 'Primary_premise': 'INTERVENTION 1:    Diagnostic (FLT PET)   Patients with early stage, ER positive primary breast cancer undergo FLT PET scan at baseline and 1-6 weeks after the start of standard endocrine treatment. The surgery follows 1-7 days after the second FLT PET scan.   Tracer used in the FLT PET (positron emission tomography) scanning procedure: [F18] fluorothymidine.   Positron Emission Tomography: Undergo FLT PET   Laboratory Biomarker Analysis: Correlative studies - Ki67 staining of the tumor tissue in the biopsy and surgical specimen.',
 'Secondary_premise': 'INTERVENTION 1:    Arm A   Patients receive oral capecitabine twice daily on days

In [11]:
def train_eval_model(epochs, epochs_to_save, learning_rate, results_location, device, model_name):
    os.makedirs(results_location, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    train_dataset = preprocess_data(tokenizer, train_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)
    dev_dataset = preprocess_data(tokenizer, dev_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)
    test_dataset = preprocess_data(tokenizer, test_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)

    # Initialize the data collator
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

    # Create DataLoader with DataCollator
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
    dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

    model = get_model(model_name, device)
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    num_training_steps = epochs * len(train_loader)
    lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    clasification_reports_dev, clasification_reports_test = {}, {}

    best_test_predictions, best_test_f1 = None, 0
    for epoch in tqdm(range(epochs), desc="Training Epochs"):
        train(model, train_loader, optimizer, lr_scheduler, device)
        if epoch in epochs_to_save:
            train_accuracy, train_f1, train_predictions, dev_labels = evaluate(model, train_loader, "Train", device)
            dev_accuracy, dev_f1, dev_predictions, dev_labels = evaluate(model, dev_loader, "Dev", device)
            test_accuracy, test_f1, test_predictions, test_labels = evaluate(model, test_loader, "Test", device)

            print(f"Epoch {epoch} - Train Accuracy: {train_accuracy:0.5f}, \t Train F1: {train_f1:0.5f}")
            print(f"Epoch {epoch} - Dev   Accuracy: {dev_accuracy:0.5f}, \t Dev   F1: {dev_f1:0.5f}")
            print(f"Epoch {epoch} - Test  Accuracy: {test_accuracy:0.5f}, \t Test  F1: {test_f1:0.5f}")
            clasification_reports_dev[epoch] = classification_report(dev_labels, dev_predictions, output_dict=True, target_names=['Contradiction', 'Entailment'], zero_division=0)
            clasification_reports_test[epoch] = classification_report(test_labels, test_predictions, output_dict=True, target_names=['Contradiction', 'Entailment'], zero_division=0)

            dev_save_path = os.path.join(results_location, f"dev_predictions_epoch{epoch}_{premise_prefix}.json")
            test_save_path = os.path.join(results_location, f"test_predictions_epoch{epoch}_{premise_prefix}.json")
            model_save_path = os.path.join(results_location, f"model_optimizer_epoch{epoch}_{premise_prefix}.pt")

            save_predictions(dev_data, dev_predictions, dev_save_path)
            save_predictions(test_data, test_predictions, test_save_path)
            save_model_and_optimizer(model, optimizer, model_save_path)

            if test_f1 > best_test_f1:
                best_test_f1 = test_f1
                best_test_predictions = test_predictions
    return best_test_f1, best_test_predictions
    

In [12]:
import nltk
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Helper function to prepare data for vectorization
def prepare_data(data):
    combined_texts = []
    for key, value in data.items():
        premise = value['Primary_premise']
        if 'Secondary_premise' in value:
            premise += ' ' + value['Secondary_premise']
        combined_texts.append(premise)
    return combined_texts

# Prepare the train, dev, and test data
train_texts = prepare_data(train_data)
dev_texts = prepare_data(dev_data)
test_texts = prepare_data(test_data)

# Create and fit the TF-IDF vectorizer on the training data
vectorizer = TfidfVectorizer()
vectorizer.fit(train_texts)

# Helper function to summarize a text
def summarize_text(text, vectorizer, max_words=256):
    sentences = nltk.sent_tokenize(text)
    if len(sentences) == 1:
        return sentences[0]

    tfidf_matrix = vectorizer.transform(sentences)
    sentence_scores = np.sum(tfidf_matrix.toarray(), axis=1)

    num_sentences = int(len(sentences) * 0.3)
    top_sentence_indices = np.argsort(sentence_scores)[-num_sentences:]
    top_sentences = [sentences[i] for i in sorted(top_sentence_indices)]

    summary = ' '.join(top_sentences)
    summary_words = summary.split(' ')
    if len(summary_words) > max_words:
        summary = ' '.join(summary_words[:max_words])

    return summary

# Function to process and summarize data
def process_and_summarize(data, vectorizer):
    for key, value in data.items():
        premise = value['Primary_premise']
        if 'Secondary_premise' in value:
            premise += ' ' + value['Secondary_premise']

        summary = summarize_text(premise, vectorizer)
        data[key]['Extractive_Primary_premise'] = summary

# Process and summarize each dataset
process_and_summarize(train_data, vectorizer)
process_and_summarize(dev_data, vectorizer)
process_and_summarize(test_data, vectorizer)

In [13]:
with open(f'test_final.json', 'r') as f:
    test_data = json.load(f)

experiments_f1_dict = {'Pred_T5_Combined_0': 0.612,
                        'Pred_T5_Separate_0': 0.6368829049367606,
                        'Pred_T5_Separate_2': 0.6311501699916607,
                        'Pred_T5_Separate_5': 0.6152551339393064,
                        'Pred_T5_Separate_7': 0.5177701692626706,
                        'Pred_T5_Separate_10': 0.617961796179618,
                        'Pred_Truncated': 0.503923103529883,
                        'Pred_Extractive': 0.5458404074702886}

In [14]:
test_data[list(test_data.keys())[0]].keys()

dict_keys(['Type', 'Section_id', 'Primary_id', 'Secondary_id', 'Statement', 'Label', 'Primary_premise', 'Secondary_premise', 'Summarized_Primary_premise_0', 'Summarized_Secondary_premise_0', 'Summarized_Primary_premise_2', 'Summarized_Secondary_premise_2', 'Summarized_Primary_premise_5', 'Summarized_Secondary_premise_5', 'Summarized_Primary_premise_7', 'Summarized_Secondary_premise_7', 'Summarized_Primary_premise_10', 'Summarized_Secondary_premise_10', 'Scifive_Primary_premise_0', 'Scifive_Seconday_premise_0', 'Scifive_Primary_premise_2', 'Scifive_Seconday_premise_2', 'Scifive_Primary_premise_5', 'Scifive_Seconday_premise_5', 'Scifive_Primary_premise_7', 'Scifive_Seconday_premise_7', 'Combined_Primary_premise_0', 'Pred_T5_Combined_0', 'Pred_T5_Separate_0', 'Pred_T5_Separate_2', 'Pred_T5_Separate_5', 'Pred_T5_Separate_7', 'Pred_T5_Separate_10', 'Pred_Truncated', 'Extractive_Primary_premise', 'Pred_Extractive'])

In [15]:
# experiments_f1_dict = {}

for premise_prefix in ["Combined_", "Summarized_", "", "Extractive_", "Scifive_"]:
    for fine_tuning_steps_suffix in ["_0", "_2", "_5", "_7", "_10", ""]:
        if premise_prefix == "Combined_" and fine_tuning_steps_suffix != "_0":
            continue
        if premise_prefix == "Scifive_" and (fine_tuning_steps_suffix == "_10" or fine_tuning_steps_suffix == ""):
            continue
        if premise_prefix == "" and fine_tuning_steps_suffix != "":
            continue
        if premise_prefix == "Summarized_" and fine_tuning_steps_suffix == "":
            continue
        if premise_prefix == "Extractive_" and fine_tuning_steps_suffix != "":
            continue
        
        premise_combined = premise_prefix == "Combined_" or premise_prefix == "Extractive_" or premise_prefix == "Scifive_"
        label_name = "Pred_"
        if premise_prefix == "Combined_":
            label_name += f"T5_Combined{fine_tuning_steps_suffix}" 
        elif premise_prefix == "Scifive_":
            label_name += f"SciFive{fine_tuning_steps_suffix}"
        elif premise_prefix == "Summarized_":
            label_name += f"T5_Separate{fine_tuning_steps_suffix}"
        elif premise_prefix == "Extractive_":
            label_name += f"Extractive"
        else:
            label_name += f"Truncated"

        print(label_name)

        if label_name in experiments_f1_dict:
            continue

        primary_premise_to_use = premise_prefix + "Primary_premise" + fine_tuning_steps_suffix
        secondary_premise_to_use = premise_prefix + "Secondary_premise" + fine_tuning_steps_suffix

        epochs = 40 #if not(premise_combined) else 20
        epochs_to_save = np.arange(epochs)
        learning_rate = 4e-5 if not(premise_combined) else 5e-5
        batch_size = 16 if premise_prefix != "Scifive_" else 16
        if "Truncated" in label_name:
            batch_size = 12
            print(f"batch_size: {batch_size}")
            learning_rate = 5e-5
        if premise_prefix == "Summarized_":
            learning_rate = 5e-5
        results_location = "results"  # Replace with your location
        device = DEVICE
        model_name = MODEL_NAME

        best_test_f1, best_test_predictions = train_eval_model(epochs, epochs_to_save, learning_rate, results_location, device, model_name)

        experiments_f1_dict[label_name] = best_test_f1

        print(f"Best Test F1: {best_test_f1:0.5f}")
        for key_idx, key in enumerate(test_data):
            test_data[key][label_name] = "Entailment" if best_test_predictions[key_idx] else "Contradiction"
        
        with open(f'test_final.json', 'w') as f:
            json.dump(test_data, f)

Pred_T5_Combined_0
Pred_T5_Separate_0
Pred_T5_Separate_2
Pred_T5_Separate_5
Pred_T5_Separate_7
Pred_T5_Separate_10
Pred_Truncated
Pred_Extractive
Pred_SciFive_0


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-v3-base and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.42it/s]


Epoch 0 - Train Accuracy: 0.50000, 	 Train F1: 0.33333
Epoch 0 - Dev   Accuracy: 0.50000, 	 Dev   F1: 0.33333
Epoch 0 - Test  Accuracy: 0.50000, 	 Test  F1: 0.33333


Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.75it/s]


Epoch 1 - Train Accuracy: 0.50000, 	 Train F1: 0.33333
Epoch 1 - Dev   Accuracy: 0.50000, 	 Dev   F1: 0.33333
Epoch 1 - Test  Accuracy: 0.50000, 	 Test  F1: 0.33333


...

Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.02it/s]


Epoch 39 - Train Accuracy: 0.50000, 	 Train F1: 0.33333
Epoch 39 - Dev   Accuracy: 0.50000, 	 Dev   F1: 0.33333
Epoch 39 - Test  Accuracy: 0.50000, 	 Test  F1: 0.33333


Training Epochs: 100%|██████████| 40/40 [20:57<00:00, 31.45s/it]


Best Test F1: 0.45132
Pred_SciFive_2


c:\Users\shahr\anaconda3\envs\lm-prompt-turbo2\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-v3-base and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-s

In [17]:
experiments_f1_dict

{'Pred_T5_Combined_0': 0.612,
 'Pred_T5_Separate_0': 0.6368829049367606,
 'Pred_T5_Separate_2': 0.6311501699916607,
 'Pred_T5_Separate_5': 0.6152551339393064,
 'Pred_T5_Separate_7': 0.5177701692626706,
 'Pred_T5_Separate_10': 0.617961796179618,
 'Pred_Truncated': 0.503923103529883,
 'Pred_Extractive': 0.5458404074702886}

In [11]:
fine_tuning_steps_suffix = "_0"  # "_0" or "_2" or "_5" or "_7" or "_10" or ""
premise_prefix = "Combined_" # "Summarized_" or ""Scifive_" or "" or "Combined_"
premise_combined = premise_prefix == "Combined_"

epochs = 40
epochs_to_save = np.arange(epochs)
batch_size = 16
learning_rate = 4e-5
results_location = "results"  # Replace with your location
device = DEVICE
model_name = MODEL_NAME

primary_premise_to_use = premise_prefix + "Primary_premise" + fine_tuning_steps_suffix
secondary_premise_to_use = premise_prefix + "Secondary_premise" + fine_tuning_steps_suffix

In [12]:
os.makedirs(results_location, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_dataset = preprocess_data(tokenizer, train_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)
dev_dataset = preprocess_data(tokenizer, dev_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)
test_dataset = preprocess_data(tokenizer, test_data, premise_combined, primary_premise_to_use, secondary_premise_to_use)

# Initialize the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Create DataLoader with DataCollator
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
dev_loader = DataLoader(dev_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

model = get_model(model_name, device)
optimizer = AdamW(model.parameters(), lr=learning_rate)

c:\Users\shahr\anaconda3\envs\lm-prompt-turbo2\lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at cross-encoder/nli-deberta-v3-base and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-s

In [13]:
print(MODEL_NAME)

cross-encoder/nli-deberta-v3-base


In [14]:
num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
clasification_reports_dev, clasification_reports_test = {}, {}

for epoch in tqdm(range(epochs), desc="Training Epochs"):
    train(model, train_loader, optimizer, lr_scheduler, device)
    if epoch in epochs_to_save:
        train_accuracy, train_f1, train_predictions, dev_labels = evaluate(model, train_loader, "Train", device)
        dev_accuracy, dev_f1, dev_predictions, dev_labels = evaluate(model, dev_loader, "Dev", device)
        test_accuracy, test_f1, test_predictions, test_labels = evaluate(model, test_loader, "Test", device)

        print(f"Epoch {epoch} - Train Accuracy: {train_accuracy:0.5f}, \t Train F1: {train_f1:0.5f}")
        print(f"Epoch {epoch} - Dev   Accuracy: {dev_accuracy:0.5f}, \t Dev   F1: {dev_f1:0.5f}")
        print(f"Epoch {epoch} - Test  Accuracy: {test_accuracy:0.5f}, \t Test  F1: {test_f1:0.5f}")
        clasification_reports_dev[epoch] = classification_report(dev_labels, dev_predictions, output_dict=True, target_names=['Contradiction', 'Entailment'], zero_division=0)
        clasification_reports_test[epoch] = classification_report(test_labels, test_predictions, output_dict=True, target_names=['Contradiction', 'Entailment'], zero_division=0)

        dev_save_path = os.path.join(results_location, f"dev_predictions_epoch{epoch}_{premise_prefix}.json")
        test_save_path = os.path.join(results_location, f"test_predictions_epoch{epoch}_{premise_prefix}.json")
        model_save_path = os.path.join(results_location, f"model_optimizer_epoch{epoch}_{premise_prefix}.pt")

        save_predictions(dev_data, dev_predictions, dev_save_path)
        save_predictions(test_data, test_predictions, test_save_path)
        save_model_and_optimizer(model, optimizer, model_save_path)

Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.92it/s]


Epoch 0 - Train Accuracy: 0.53471, 	 Train F1: 0.51705
Epoch 0 - Dev   Accuracy: 0.54000, 	 Dev   F1: 0.51431
Epoch 0 - Test  Accuracy: 0.55200, 	 Test  F1: 0.53092


Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.90it/s]


Epoch 1 - Train Accuracy: 0.50412, 	 Train F1: 0.34644
Epoch 1 - Dev   Accuracy: 0.50000, 	 Dev   F1: 0.33333
Epoch 1 - Test  Accuracy: 0.50000, 	 Test  F1: 0.33333


Evaluating on Test: 100%|██████████| 32/32 [00:02<00:00, 15.71it/s]


...

Evaluating on Test: 100%|██████████| 32/32 [00:01<00:00, 17.17it/s]


Epoch 38 - Train Accuracy: 0.99176, 	 Train F1: 0.99176
Epoch 38 - Dev   Accuracy: 0.60500, 	 Dev   F1: 0.60380
Epoch 38 - Test  Accuracy: 0.61600, 	 Test  F1: 0.61526


Evaluating on Test: 100%|██████████| 32/32 [00:01<00:00, 17.20it/s]


Epoch 39 - Train Accuracy: 0.99176, 	 Train F1: 0.99176
Epoch 39 - Dev   Accuracy: 0.60000, 	 Dev   F1: 0.59855
Epoch 39 - Test  Accuracy: 0.61800, 	 Test  F1: 0.61732


Training Epochs: 100%|██████████| 40/40 [20:17<00:00, 30.44s/it]


In [15]:
print("Classification Reports for Test Set")
for  epoch in clasification_reports_test.keys():
    print(clasification_reports_test[epoch])
    print()

Classification Reports for Test Set
{'Contradiction': {'precision': 0.5365168539325843, 'recall': 0.764, 'f1-score': 0.6303630363036304, 'support': 250}, 'Entailment': {'precision': 0.5902777777777778, 'recall': 0.34, 'f1-score': 0.43147208121827413, 'support': 250}, 'accuracy': 0.552, 'macro avg': {'precision': 0.563397315855181, 'recall': 0.552, 'f1-score': 0.5309175587609523, 'support': 500}, 'weighted avg': {'precision': 0.563397315855181, 'recall': 0.552, 'f1-score': 0.5309175587609523, 'support': 500}}

{'Contradiction': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 250}, 'Entailment': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 250}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 500}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 500}}

{'Contradiction': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 250}, '